In [ ]:
import time
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# 1. THE MOCK BACKEND LOGIC (Unchanged)
def search_engine(query):
    database = [
        {"id": 1, "type": "article", "title": "Optimizing SQL Queries", "tags": ["Database", "Performance"], "snippet": "Use EXPLAIN ANALYZE to understand query plans. Avoid SELECT * in production..."},
        {"id": 2, "type": "article", "title": "Intro to Kubernetes", "tags": ["DevOps", "K8s"], "snippet": "A 10-minute crash course on Pods, Services, and Ingress controllers..."},
        {"id": 3, "type": "article", "title": "Python 3.11 Features", "tags": ["Python", "Updates"], "snippet": "Significant speed improvements and better error messages in tracebacks..."},
        {"id": 4, "type": "article", "title": "React Hooks Patterns", "tags": ["Frontend", "React"], "snippet": "Understanding useMemo and useCallback to prevent unnecessary re-renders..."},
    ]
    time.sleep(0.5) # Reduced latency slightly for snappier feel
    if not query: return database
    query = query.lower()
    return [doc for doc in database if query in doc['title'].lower() or query in doc['snippet'].lower()]

# 2. BEAUTIFIED CSS
style = """
<style>
    :root {
        --brand: #3b82f6;         /* Bright Blue */
        --brand-light: #eff6ff;   /* Very light blue */
        --text-main: #1e293b;     /* Dark Slate */
        --text-sub: #64748b;      /* Muted Slate */
        --bg-card: #ffffff;
        --border: #e2e8f0;
    }

    /* Main Container Font */
    .app-container {
        font-family: 'Inter', -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
        max-width: 800px; /* Constrain width for readability */
        margin: 0 auto;
    }

    /* --- NEW: Header Styles --- */
    .app-header {
        text-align: center;
        margin-bottom: 25px;
        padding-bottom: 15px;
        border-bottom: 1px solid var(--border);
    }
    .app-title {
        font-size: 1.8rem;
        font-weight: 700;
        color: var(--text-main);
        margin: 0;
        letter-spacing: -0.02em;
    }
    .app-subtitle {
        font-size: 0.9rem;
        color: var(--text-sub);
        margin-top: 5px;
    }

    /* --- Card Styles --- */
    .result-card {
        background: var(--bg-card);
        border: 1px solid var(--border);
        border-radius: 8px;
        padding: 16px;
        margin-bottom: 12px;
        transition: all 0.2s ease;
        box-shadow: 0 1px 3px rgba(0,0,0,0.05);
        border-left: 4px solid transparent;
    }
    
    .result-card:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
        border-left: 4px solid var(--brand);
        border-color: #cbd5e1;
    }

    .card-top-row {
        display: flex;
        justify-content: space-between;
        align-items: center;
        margin-bottom: 6px;
    }

    .card-type {
        font-size: 0.7rem;
        font-weight: 700;
        text-transform: uppercase;
        letter-spacing: 0.05em;
        color: var(--brand);
        background: var(--brand-light);
        padding: 2px 8px;
        border-radius: 12px;
    }

    .card-title {
        font-size: 1.1rem;
        font-weight: 600;
        color: var(--text-main);
        margin-bottom: 6px;
    }
    
    .card-snippet {
        font-size: 0.9rem;
        color: var(--text-sub);
        line-height: 1.5;
        margin-bottom: 10px;
    }

    .tag {
        font-size: 0.75rem;
        background: #f1f5f9;
        color: #475569;
        padding: 2px 8px;
        border-radius: 4px;
        border: 1px solid #e2e8f0;
        margin-right: 5px;
    }
</style>
"""
display(HTML(style))

In [5]:
class SearchApp:
    def __init__(self):
        # -- 1. HEADER WIDGET --
        self.header = widgets.HTML(
            value="""
            <div class="app-container">
                <div class="app-header">
                    <div class="app-title">⚡ Knowledge Base</div>
                    <div class="app-subtitle">Search documentation, videos, and articles instantly</div>
                </div>
            </div>
            """
        )

        # -- 2. SEARCH INPUTS --
        self.search_box = widgets.Text(
            placeholder='e.g., "Kubernetes" or "SQL"',
            layout=widgets.Layout(width='60%', height='45px'),
            continuous_update=False 
        )
        
        self.search_btn = widgets.Button(
            description='Search',
            icon='search',
            button_style='primary', # Blue button
            layout=widgets.Layout(width='120px', height='45px')
        )
        
        self.out = widgets.Output()
        
        # -- 3. LAYOUT ASSEMBLY --
        # We wrap inputs in an HBox with some margin for spacing
        input_area = widgets.HBox(
            [self.search_box, self.search_btn], 
            layout=widgets.Layout(justify_content='center', margin='0 0 30px 0', align_items='center')
        )
        
        self.container = widgets.VBox([
            self.header,
            input_area,
            self.out
        ])
        
        # -- 4. EVENT BINDINGS --
        self.search_btn.on_click(self.on_search)
        self.search_box.observe(self.on_search, names='value')

    def get_icon(self, doc_type):
        # Helper to add icons based on type
        icons = {
            "article": "📄",
            "video": "🎥",
            "doc": "📘"
        }
        return icons.get(doc_type, "📎")

    def render_results(self, results):
        # Note: We wrap everything in .app-container to center it
        html_content = '<div class="app-container">'
        
        if not results:
            html_content += """
                <div style="text-align: center; color: #94a3b8; padding: 40px; border: 2px dashed #e2e8f0; border-radius: 8px;">
                    <div style="font-size: 2rem; margin-bottom: 10px;">🔍</div>
                    <h3>No results found</h3>
                    <p>Try using broader keywords.</p>
                </div>
            """
        else:
            html_content += f'<div style="margin-bottom: 15px; color: #64748b; font-size: 0.85rem;">Found {len(results)} matches</div>'
            
            for doc in results:
                icon = self.get_icon(doc['type'])
                tags_html = "".join([f'<span class="tag">{t}</span>' for t in doc['tags']])
                
                html_content += f"""
                <div class="result-card">
                    <div class="card-top-row">
                        <span class="card-type">{icon} {doc['type']}</span>
                        <span style="color:#cbd5e1; font-size:0.8rem">#{doc['id']}</span>
                    </div>
                    <div class="card-title">{doc['title']}</div>
                    <div class="card-snippet">{doc['snippet']}</div>
                    <div class="card-tags">{tags_html}</div>
                </div>
                """
        
        html_content += '</div>'
        return html_content

    def on_search(self, *args):
        query = self.search_box.value
        
        with self.out:
            clear_output(wait=True)
            # improved loader
            display(HTML("""
                <div style="text-align:center; padding: 40px; color: #64748b;">
                    <span style="display:inline-block; animation: spin 1s linear infinite;">⏳</span> Searching database...
                    <style>@keyframes spin { 100% { -webkit-transform: rotate(360deg); transform:rotate(360deg); } }</style>
                </div>
            """))
            
            results = search_engine(query)
            
            clear_output(wait=True)
            display(HTML(self.render_results(results)))

    def display(self):
        display(self.container)

In [6]:
app = SearchApp()
app.display()